# Quality Controlling a generic data object

### Objective:

This notebook shows how to use CoTeDe with a netcdf file (using xarray), or a pandas dataframe.

In [76]:
import cotede
print("CoTeDe version: {}".format(cotede.__version__))

CoTeDe version: 0.20.2


In [77]:
# install xarray/pandas
!pip install xarray

In [79]:
# Import requirements

from datetime import datetime

import numpy as np
from numpy import ma
from cotede.qc import ProfileQC
import pandas as pd
import xarray as xr

# xarray/netcdf

In [80]:
# load up data in xarray from netcdf file
mydata = xr.open_dataset('gak.nc')
mydata

<xarray.Dataset>
Dimensions:                       (time: 45, z: 1)
Coordinates:
    latitude                      float64 ...
    longitude                     float64 ...
  * time                          (time) datetime64[ns] 2000-02-21T00:17:00 ... 2000-02-28T06:17:00
  * z                             (z) float64 0.0
Data variables:
    station                       object ...
    crs                           int32 ...
    sea_water_practical_salinity  (time, z) float64 ...
    sea_water_temperature         (time, z) float64 ...
Attributes:
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3, NCCSV-1.0
    date_created:                  2020-02-29T04:10:00Z
    featureType:                   TimeSeries
    cdm_data_type:                 TimeSeries
    cdm_timeseries_variables:      air_temperature,air_pressure,dissolved_car...
    contributor_email:             uaf.web@alaska.edu,feedback@axiomdatascien...
    contributor_name:              University Of Alaska Fairbanks (UAF),Axiom...
    contributor_role:              contributor,processor
    contributor_role_vocabulary:   CI_RoleCode
    contributor_url:               http://www.uaf.edu/,https://www.axiomdatas...
    creator_country:               USA
    creator_email:                 dana.greeley@noaa.gov
    creator_institution:           NOAA Pacific Marine Environmental Lab (PMEL)
    creator_name:                  NOAA Pacific Marine Environmental Lab (PMEL)
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   https://www.pmel.noaa.gov/co2/
    Easternmost_Easting:           -149.5
    geospatial_lat_max:            59.85
    geospatial_lat_min:            59.85
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -149.5
    geospatial_lon_min:            -149.5
    geospatial_lon_units:          degrees_east
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from NOAA Pacific Marine Enviro...
    id:                            14118
    infoUrl:                       https://portal.aoos.org/#metadata/14118/st...
    institution:                   NOAA Pacific Marine Environmental Lab (PMEL)
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         59.85
    platform:                      fixed
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    publisher_country:             USA
    publisher_email:               dana.greeley@noaa.gov
    publisher_institution:         NOAA Pacific Marine Environmental Lab (PMEL)
    publisher_name:                NOAA Pacific Marine Environmental Lab (PMEL)
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 https://www.pmel.noaa.gov/co2/
    references:                    https://www.pmel.noaa.gov/co2/story/GAKOA,,
    sourceUrl:                     https://sensors.axds.co/api/
    Southernmost_Northing:         59.85
    standard_name_vocabulary:      CF Standard Name Table v71
    summary:                       Timeseries data from 'Gulf of Alaska Ocean...
    time_coverage_end:             2020-02-28T06:17:00Z
    time_coverage_start:           2020-02-21T00:17:00Z
    title:                         Gulf of Alaska Ocean Acidification Mooring
    Westernmost_Easting:           -149.5

In [81]:
# define qc config
cfg = {
    'main': {
        'valid_datetime': None,
    },
    'sea_water_temperature': {
        'global_range': {
            'minval': 4.0,
            'maxval': 5.0,
        },
    },
    'sea_water_practical_salinity': {
        'global_range': {
            'minval': 29.0,
            'maxval': 32.0,
        },
    },
}
cfg

{'main': {'valid_datetime': None},
 'sea_water_temperature': {'global_range': {'minval': 4.0, 'maxval': 5.0}},
 'sea_water_practical_salinity': {'global_range': {'minval': 29.0,
   'maxval': 32.0}}}

In [82]:
# run qc
pqced = ProfileQC(mydata, cfg=cfg, attributes={})

/home/jaustin/apps/anaconda3/envs/cotede37/lib/python3.7/site-packages/cotede/qctests/global_range.py:39: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  flag[np.nonzero(feature < minval)] = self.flag_bad
/home/jaustin/apps/anaconda3/envs/cotede37/lib/python3.7/site-packages/cotede/qctests/global_range.py:40: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  flag[np.nonzero(feature > maxval)] = self.flag_bad
/home/jaustin/apps/anaconda3/envs/cotede37/lib/python3.7/site-packages/cotede/qctests/global_range.py:42: FutureWarning: Using a non-tuple

In [83]:
# inspect flags
pqced.flags.keys()

dict_keys(['common', 'sea_water_practical_salinity', 'sea_water_temperature'])

In [84]:
pqced.flags['sea_water_temperature'].keys()

dict_keys(['valid_datetime', 'global_range', 'overall'])

In [85]:
pqced.flags['sea_water_temperature']['global_range'].transpose()

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 4, 4, 4, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1]], dtype=int8)

In [73]:
pqced.flags['sea_water_temperature']['overall'].transpose()

array([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 4, 4, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3]], dtype=int8)

# pandas dataframe

In [89]:
# load up data in pandas from csv file
df = pd.read_csv('gak.csv', skiprows=[1])
df.head()

,time,latitude,longitude,z,sea_water_practical_salinity,sea_water_temperature
0,2020-02-28T06:17:00Z,59.85,-149.5,0.0,31.31,4.09
1,2020-02-28T03:17:00Z,59.85,-149.5,0.0,31.41,4.29
2,2020-02-28T00:17:00Z,59.85,-149.5,0.0,31.52,4.49
3,2020-02-27T21:17:00Z,59.85,-149.5,0.0,31.55,4.53
4,2020-02-27T18:17:00Z,59.85,-149.5,0.0,31.55,4.52


In [90]:
# run qc
pqced_df = ProfileQC(df, cfg=cfg, attributes={})

ValueError: Length of passed values is 1, index implies 45.